In [ ]:
import random

class Player:
    def __init__(self, name=None):
        self.name = name if name is not None else self.__class__.__name__
        self.deck = list(range(1, 7))
        self.stack = [0]
    
    def reset(self):
        """Reset player to initial state"""
        self.deck = list(range(1, 7))
        self.stack = [0]
    
    def get_draw(self):
        """Shuffle deck and return 2 cards"""
        random.shuffle(self.deck)
        return self.deck[:2]
    
    def get_available_plays(self, draw):
        """Get available safe and risky plays from the draw"""
        top = self.stack[-1]
        safe_play = top + 1 if top + 1 in draw else None
        risky_play = top + 2 if top + 2 in draw else None
        return safe_play, risky_play
    
    def play_card(self, card):
        """Play a card from hand to stack"""
        self.stack.append(card)
        self.deck.remove(card)
        return f"played {card}"
    
    def check_opponent(self, opponent_stack):
        """Check if opponent can be reset"""
        if len(opponent_stack) >= 2 and abs(opponent_stack[-1] - opponent_stack[-2]) == 2:
            return "check_success"
        else:
            return "check_fail"
    
    def can_declare_win(self):
        """Check if player can declare victory (stack top is 6)"""
        return self.stack[-1] == 6
    
    def declare_win(self):
        """Declare victory"""
        return "declare_win"
    
    def make_move(self, opponent_stack):
        """Override this method in subclasses to implement different strategies"""
        raise NotImplementedError

class BasicPlayer(Player):
    """Basic strategy: Safe first, then risky, then check"""
    def make_move(self, opponent_stack):
      
        if self.can_declare_win():
            return self.declare_win()
            
        draw = self.get_draw()
        safe_play, risky_play = self.get_available_plays(draw)
        
        if safe_play:
            return self.play_card(safe_play)
        elif risky_play:
            return self.play_card(risky_play)
        else:
            return self.check_opponent(opponent_stack)

class AggressivePlayer(Player):
    """Always plays risky when possible, then safe, then check"""
    def make_move(self, opponent_stack):
   
        if self.can_declare_win():
            return self.declare_win()
            
        draw = self.get_draw()
        safe_play, risky_play = self.get_available_plays(draw)
        
        if risky_play:
            return self.play_card(risky_play)
        elif safe_play:
            return self.play_card(safe_play)
        else:
            return self.check_opponent(opponent_stack)

class CautiousPlayer(Player):
    """Only plays safe moves, checks more often"""
    def make_move(self, opponent_stack):
        
        if self.can_declare_win():
            return self.declare_win()
            
        draw = self.get_draw()
        safe_play, risky_play = self.get_available_plays(draw)
        
        if safe_play:
            return self.play_card(safe_play)
        else:
            return self.check_opponent(opponent_stack)

def play_game(player1, player2, verbose=False):
    """Play a single game between two players"""
    player1.reset()
    player2.reset()
    
    turn = 1
    turn_count = 0
    
    while True:
        turn_count += 1
        current_player = player1 if turn == 1 else player2
        opponent = player2 if turn == 1 else player1
        
        action = current_player.make_move(opponent.stack)
        
        if action == "declare_win":
            if verbose:
                print(f"{current_player.name} declares victory and wins!")
            return turn
        elif action == "check_success":
            opponent.deck.extend(opponent.stack[1:])
            opponent.stack = [0]
        
        if verbose:
            print(f"Turn {turn_count} - {current_player.name}: {action}")
            print(f"  {player1.name} - Stack: {player1.stack}, Deck: {player1.deck}")
            print(f"  {player2.name} - Stack: {player2.stack}, Deck: {player2.deck}")
            print()
       
        turn = 2 if turn == 1 else 1
        
        if turn_count > 1000:
            if verbose:
                print("Game too long, declaring draw")
            return 0  

def simulate_matchup(player1_class, player2_class, verbose=False, n_games=10000):
    """Simulate many games between two player types"""
    player1 = player1_class()
    player2 = player2_class()
    
    wins = [0, 0, 0]  
    
    for _ in range(n_games):
        result = play_game(player1, player2)
        wins[result] += 1
    
    if verbose:
        print(f"Matchup: {player1.name} vs {player2.name}")
        print(f"Games played: {n_games}")
        print(f"Results: {player1.name} {wins[1]} wins, {player2.name} {wins[2]} wins, {wins[0]} draws")
        print(f"Win rates: {player1.name} {wins[1]/n_games:.3f}, {player2.name} {wins[2]/n_games:.3f}")
        print()
    
    return {
        'player1_wins': wins[1],
        'player2_wins': wins[2], 
        'draws': wins[0],
        'player1_winrate': wins[1] / n_games,
        'player2_winrate': wins[2] / n_games
    }

In [ ]:
print("=== Strategy Comparison ===")

print("=== Basic Player Matchups ===")
result1 = simulate_matchup(BasicPlayer, AggressivePlayer, verbose=True, n_games=10000)

result2 = simulate_matchup(BasicPlayer, CautiousPlayer, verbose=True, n_games=10000)

result3 = simulate_matchup(BasicPlayer, BasicPlayer, verbose=True, n_games=10000)

print("=== Aggressive Player Matchups ===")
result4 = simulate_matchup(AggressivePlayer, BasicPlayer, verbose=True, n_games=10000)

result5 = simulate_matchup(AggressivePlayer, CautiousPlayer, verbose=True, n_games=10000)

result6 = simulate_matchup(AggressivePlayer, AggressivePlayer, verbose=True, n_games=10000)

print("=== Cautious Player Matchups ===")
result7 = simulate_matchup(CautiousPlayer, BasicPlayer, verbose=True, n_games=10000)

result8 = simulate_matchup(CautiousPlayer, AggressivePlayer, verbose=True, n_games=10000)

result9 = simulate_matchup(CautiousPlayer, CautiousPlayer, verbose=True, n_games=10000)


=== Strategy Comparison ===
=== Basic Player Matchups ===
Matchup: BasicPlayer vs AggressivePlayer
Games played: 10000
Results: BasicPlayer 4634 wins, AggressivePlayer 5366 wins, 0 draws
Win rates: BasicPlayer 0.463, AggressivePlayer 0.537

Matchup: BasicPlayer vs CautiousPlayer
Games played: 10000
Results: BasicPlayer 6779 wins, CautiousPlayer 3221 wins, 0 draws
Win rates: BasicPlayer 0.678, CautiousPlayer 0.322

Matchup: BasicPlayer vs BasicPlayer
Games played: 10000
Results: BasicPlayer 5489 wins, BasicPlayer 4511 wins, 0 draws
Win rates: BasicPlayer 0.549, BasicPlayer 0.451

=== Aggressive Player Matchups ===
Matchup: AggressivePlayer vs BasicPlayer
Games played: 10000
Results: AggressivePlayer 6185 wins, BasicPlayer 3815 wins, 0 draws
Win rates: AggressivePlayer 0.619, BasicPlayer 0.382

Matchup: AggressivePlayer vs CautiousPlayer
Games played: 10000
Results: AggressivePlayer 5813 wins, CautiousPlayer 4187 wins, 0 draws
Win rates: AggressivePlayer 0.581, CautiousPlayer 0.419

Matc